In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:

filename = "yelp-100K/testing_set_seed123_yelp100k.csv"
test_set = pd.read_csv(f"../data/{filename}",sep=",",
    header=None,
    names=["userID", "itemID", "Rating"],)
test_set



In [ ]:
all_users_tp = test_set.groupby('userID')['itemID'].apply(list).tolist()
all_users_tp.__len__()

In [6]:
neumf = "neumf/reco_matrix_neumf_yelp100k_100.npy"
mf = "mf/reco_matrix_mf_yelp100k_100.npy"
wmf = "wmf/reco_matrix_wmf_yelp100k_100.npy"
vaecf = "vaecf/reco_matrix_vaecf_yelp100k_100.npy"
reco_matrix = np.load(f"../model_reco/yelp/{mf}")

In [7]:
reco_matrix = reco_matrix[0] # if reco matrix is inside a nested array if not use it directly

In [ ]:

from cornac.metrics import RMSE, AUC, NDCG, Precision, Recall,HitRatio

hr=HitRatio(20)
alist=[]
sum_hr=0
for u in range(reco_matrix.shape[0]):
    x=hr.compute(all_users_tp[u],reco_matrix[u][:20])
    sum_hr=sum_hr+x
plain_hr=sum_hr/reco_matrix.shape[0]

plain_hr

In [ ]:
# from NDCG import NDCG
ndcg=NDCG(20)
alist=[]
sum_ndcg=0
for u in range(reco_matrix.shape[0]):
    x=ndcg.compute(all_users_tp[u],reco_matrix[u][:20])
    sum_ndcg=sum_ndcg+x
plain_ndcg=sum_ndcg/reco_matrix.shape[0]

plain_ndcg

In [10]:

all_pds =[]
reco_matrix_fair=[]
sens="gender"
model = "mf"
# here we load the recommendations over different vaues of beta
all_pds.append(pd.read_csv(f"../results/yelp/{model}/{sens}_1.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/yelp/{model}/{sens}_2.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/yelp/{model}/{sens}_3.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/yelp/{model}/{sens}_4.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/yelp/{model}/{sens}_5.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/yelp/{model}/{sens}_6.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/yelp/{model}/{sens}_7.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/yelp/{model}/{sens}_8.csv",sep=",",))
# results/reco_mat_test.npy
for j in range(len(all_pds)):
    reco_m =[]
    for i in range(all_pds[j].__len__()):
        reco_m.append(np.array(all_pds[j].iloc[i][1:]))
    reco_matrix_fair.append(reco_m)
    



In [ ]:

hr=HitRatio(20)
hrlist=[]
for r in reco_matrix_fair:
    sum_hr=0
    for u in range(reco_matrix.shape[0]):
        x=hr.compute(all_users_tp[u],r[u])
        sum_hr=sum_hr+x
    hr_occ=sum_hr/reco_matrix.shape[0]
    hrlist.append(hr_occ)
    

[plain_hr]+hrlist

In [ ]:

hr=NDCG(20)
hrlist=[]
for r in reco_matrix_fair:
    sum_hr=0
    for u in range(reco_matrix.shape[0]):
        x=hr.compute(all_users_tp[u],r[u])
        sum_hr=sum_hr+x
    hr_occ=sum_hr/reco_matrix.shape[0]
    hrlist.append(hr_occ)
    

[plain_ndcg]+hrlist

In [14]:
restaurants = pd.read_csv(
    "../data/yelp-100K/i_id_mapping.csv",
    sep="\t",
   names=[ "item_id","Category","itemID"],
    header=0,
    # encoding="latin-1",
)
restaurants=restaurants.sort_values(by="itemID")

unique_categories = [
    "Active Life & Fitness",
    "Arts & Entertainment",
    "Automotive",
    "Bars & Nightlife",
    "Coffee,Tea & Desserts",
    "Drinks & Spirits",
    "Education & Learning",
    "Event Services",
    "Family & Kids",
    "Food & Restaurants",
    "Health & Beauty",
    "Home & Garden",
    "Miscellaneous",
    "Outdoor Activities",
    "Public Services & Community",
    "Shopping & Fashion",
    "Specialty Food & Groceries",
    "Sports & Recreation",
    "Technology & Electronics",
    "Travel & Transportation",
    "Asian",
]
for c in unique_categories:
    restaurants[c] = 0
for index, row in restaurants.iterrows():
    cats = row["Category"].split("|")
    for cat in cats:
        restaurants.at[index, cat] = 1

cat = restaurants[unique_categories]
# cat[:1]
item_features_numpy = cat.to_numpy()
item_features = {
    str(item_id): {"category_" + str(idx): value for idx, value in enumerate(row)}
    for item_id, row in enumerate(item_features_numpy)
}
# ids = list(range(0, 3416))
restaurants

users = pd.read_csv("../data/yelp-100K/u_id_mapping.csv", sep="\t",header=0,
    names=[ "user_id","Gender","userID"])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
users = users.sort_values(by="userID")
users = users[["Gender", "userID"]]
user_features_numpy = users.to_numpy()
users
item_ids = restaurants["itemID"].to_numpy()




In [17]:
import os
os.chdir("/Users/name/Desktop/work/reranking_fairnes")


In [ ]:
nusers = reco_matrix.shape[0]
new_rmatrix = np.zeros(( nusers, 20), dtype=int)
for i in range(nusers):
    new_rmatrix[i]=reco_matrix[i][:20]
new_rmatrix.shape


In [19]:
from mymetrics.GenrePrecisionMulti import GenrePrecisionMulti
from mymetrics.GenreNDCGMulti import GenreNDCGMulti
from mymetrics.GenreNDCG import GenreNDCG
####Import Metrics####

top_k = 20
###intialize them
gp = GenrePrecisionMulti(users, unique_categories, top_k)
gn = GenreNDCGMulti(users, unique_categories, top_k)
gn_old = GenreNDCG(users, unique_categories, top_k)


In [ ]:
gp_plain = gp.compute(new_rmatrix, restaurants, "Gender")
gp_plain

In [ ]:
gn_plain = gn.compute(new_rmatrix, restaurants, "Gender")
gn_plain

In [ ]:
i=0
xlist=[]
for r in reco_matrix_fair:
    i=i+1
    x=gp.compute(np.array(r), restaurants, "Gender")
    xlist.append(x[0])
    # print(x)
    print(f"{x[0]} is the bias score for b={i}")


In [ ]:
[gp_plain[0]]+xlist

In [ ]:
i=0
xlist=[]
for r in reco_matrix_fair:
    i=i+1
    x=gn.compute(np.array(r), restaurants, "Gender")
    xlist.append(x[0])
    # print(x)
    print(f"{x[0]} is the bias score for b={i}")


In [ ]:
[gn_plain[0]]+xlist